In [ ]:
import sklearn.metrics as skm
import sys
import numpy as np
from sklearn.metrics import roc_curve
from sklearn.metrics import RocCurveDisplay
import matplotlib.pyplot as plt

ssl_features_train = np.load("./old_but_full_features/self_supervised_cifar100/features_train2.npy")
ssl_labels_train = np.load("./old_but_full_features/self_supervised_cifar100/labels_train2.npy")
ssl_features_test = np.load("./old_but_full_features/self_supervised_cifar100/features_test2.npy")
ssl_features_ood = np.load("./old_but_full_features/self_supervised_cifar100/features_ood2.npy")

#ssl_features_train = np.load("./ssl_2/features_train2.npy")
#ssl_labels_train = np.load("./ssl_2/labels_train2.npy")
#ssl_features_test = np.load("./ssl_2/features_test.npy")
#ssl_features_ood = np.load("./ssl_2/features_ood.npy")


cl_features_train = np.load("./supervised_contrastive_cifar10/features_train2.npy")
cl_labels_train = np.load("./supervised_contrastive_cifar10/labels_train2.npy")
cl_features_test = np.load("./supervised_contrastive_cifar10/features_test.npy")
cl_features_ood = np.load("./supervised_contrastive_cifar10/features_ood.npy")


sp_features_train = np.load("./old_but_full_features/supervised_cifar100/features_train2.npy")
sp_labels_train = np.load("./old_but_full_features/supervised_cifar100/labels_train2.npy")
sp_features_test = np.load("./old_but_full_features/supervised_cifar100/features_test2.npy")
sp_features_ood = np.load("./old_but_full_features/supervised_cifar100/features_ood2.npy")

ds_features_train = np.load("./distillation1/features_train2.npy")
ds_labels_train = np.load("./distillation1/labels_train2.npy")
ds_features_test = np.load("./distillation1/features_test.npy")
ds_features_ood = np.load("./distillation1/features_ood.npy")


In [ ]:
# -- calculate SL and SSL score separately


import sklearn.metrics as skm
import sys
import numpy as np
from sklearn.metrics import roc_curve
from sklearn.metrics import RocCurveDisplay
import matplotlib.pyplot as plt

def get_roc_sklearn(xin, xood):
    labels = [0] * len(xin) + [1] * len(xood)
    data = np.concatenate((xin, xood))
    auroc = skm.roc_auc_score(labels, data)
    fpr,tpr,_ = roc_curve(labels, data)
    operation_idx = np.abs(tpr-0.95).argmin()
    fpr95 = fpr[operation_idx]
    return auroc, fpr95


def get_scores_multi_cluster(ftrain, ftest, food, ypred):
    print("using multiple clusters")
    xc = [ftrain[ypred == i] for i in np.unique(ypred)]
    

    din = [
        np.sum(
            (ftest - np.mean(x, axis=0, keepdims=True))
            * (
                np.linalg.pinv(np.cov(x.T, bias=True)).dot(
                    (ftest - np.mean(x, axis=0, keepdims=True)).T
                )
            ).T,
            axis=-1,
        )
        for x in xc
    ]
    dood = [
        np.sum(
            (food - np.mean(x, axis=0, keepdims=True))
            * (
                np.linalg.pinv(np.cov(x.T, bias=True)).dot(
                    (food - np.mean(x, axis=0, keepdims=True)).T
                )
            ).T,
            axis=-1,
        )
        for x in xc
    ]
#     print("din shape is looking like:", din)
    din_index = np.argmin(din, axis = 0)
    
    din = np.min(din, axis=0)
    
    dood_index = np.argmin(dood, axis = 0)
    dood = np.min(dood, axis=0)

    return din, dood, din_index, dood_index

def get_scores(ftrain, ftest, food, labelstrain, cluster):

    # standardize data
    print("ftrain size is:", ftrain.shape)
    ftrain /= np.linalg.norm(ftrain, axis=-1, keepdims=True) + 1e-10
    ftest /= np.linalg.norm(ftest, axis=-1, keepdims=True) + 1e-10
    food /= np.linalg.norm(food, axis=-1, keepdims=True) + 1e-10

    m, s = np.mean(ftrain, axis=0, keepdims=True), np.std(ftrain, axis=0, keepdims=True)

    ftrain = (ftrain - m) / (s + 1e-10)
    ftest = (ftest - m) / (s + 1e-10)
    food = (food - m) / (s + 1e-10)
    
    return get_scores_multi_cluster(ftrain, ftest, food, labelstrain)   # for supervised learning

def get_eval_results(dtest, dood):
    """
    None.
    """

    auroc, fpr95 = get_roc_sklearn(dtest, dood)
    return auroc, fpr95

In [ ]:
#calculate score of id test samples and ood test samples

# Get features --> individual cluster (self-sp and sp) --> calcualte score

ssl_din, ssl_dout, ssl_idxin, ssl_idxood = get_scores(ssl_features_train, ssl_features_test, ssl_features_ood, ssl_labels_train, cluster = 10)

sp_din, sp_dout, sp_idxin, sp_idxood = get_scores(sp_features_train, sp_features_test, sp_features_ood, sp_labels_train, cluster = 10)

cl_din, cl_dout, cl_idxin, cl_idxood = get_scores(cl_features_train, cl_features_test, cl_features_ood, cl_labels_train, cluster = 10)

ds_din, ds_dout, ds_idxin, ds_idxood = get_scores(ds_features_train, ds_features_test, ds_features_ood, ds_labels_train, cluster = 10)


In [ ]:
auroc, fpr95 = get_eval_results(ssl_din, ssl_dout)

print("AUROC is:", auroc)
print("FPR95 is:", fpr95)

auroc, fpr95 = get_eval_results(sp_din, sp_dout)

print("AUROC is:", auroc)
print("FPR95 is:", fpr95)

auroc, fpr95 = get_eval_results(cl_din, cl_dout)

print("AUROC is:", auroc)
print("FPR95 is:", fpr95)

auroc, fpr95 = get_eval_results(ds_din, ds_dout)

print("AUROC is:", auroc)
print("FPR95 is:", fpr95)


In [ ]:
np.save("./features/self_supervised_cifar100/ssl_test_score.npy",ssl_din)
np.save("./features/self_supervised_cifar100/ssl_ood_score.npy",ssl_dout)
np.save("./features/supervised_cifar100/sl_test_score.npy",sp_din)
np.save("./features/supervised_cifar100/sl_ood_score.npy",sp_dout)
np.save("./features/supervised_cifar100/sl_ood_minlabels.npy",sp_idxood)
np.save("./features/supervised_cifar100/sl_test_minlabels.npy",sp_idxin)
np.save("./features/self_supervised_cifar100/ssl_ood_minlabels.npy",ssl_idxood)
np.save("./features/self_supervised_cifar100/ssl_test_minlabels.npy",ssl_idxin)
np.save("./features/supervised_contrastive_cifar100/cl_test_score.npy",cl_din)
np.save("./features/supervised_contrastive_cifar100/cl_ood_score.npy",cl_dout)
np.save("./features/supervised_contrastive_cifar100/cl_ood_minlabels.npy",cl_idxood)
np.save("./features/supervised_contrastive_cifar100/cl_test_minlabels.npy",cl_idxin)



In [ ]:
# np.save("./old_but_full_features/self_supervised_cifar100/ssl_test_score.npy",ssl_din)
# np.save("./old_but_full_features/self_supervised_cifar100/ssl_ood_score.npy",ssl_dout)
# np.save("./old_but_full_features/supervised_cifar100/sl_test_score.npy",sp_din)
# np.save("./old_but_full_features/supervised_cifar100/sl_ood_score.npy",sp_dout)


In [ ]:
# calculate score for combined feature


import matplotlib.pyplot as plt

def dnorm(ftrain, ftest, food):
    
    print("ftrain size is:", ftrain.shape)
    ftrain /= np.linalg.norm(ftrain, axis=-1, keepdims=True) + 1e-10
    ftest /= np.linalg.norm(ftest, axis=-1, keepdims=True) + 1e-10
    food /= np.linalg.norm(food, axis=-1, keepdims=True) + 1e-10

    m, s = np.mean(ftrain, axis=0, keepdims=True), np.std(ftrain, axis=0, keepdims=True)
    print("mean value is:", np.mean(m))
    print("std value is:", np.mean(s))
#     m2, s2 = np.mean(ftest, axis=0, keepdims=True), np.std(ftest, axis=0, keepdims=True)
#     m3, s3 = np.mean(food, axis=0, keepdims=True), np.std(food, axis=0, keepdims=True)

    ftrain = (ftrain - m) / (s + 1e-10)
    ftest = (ftest - m) / (s + 1e-10)
    food = (food - m) / (s + 1e-10)
#     print("ftrain shape is:", ftrain.shape)
#     ftrain = (ftrain - m) / (s + 1e-10)
#     ftest = (ftrain - m2) / (s2 + 1e-10)
#     food = (ftrain - m3) / (s3 + 1e-10)

#     plt.figure(0).clf()
#     plt.plot(food[960])
#     plt.hist(ftrain[60], density=True, bins=100)
#     plt.show()
    
    
    return ftrain, ftest, food

#ssl_features_train,  ssl_features_test, ssl_features_ood= dnorm(ssl_features_train, ssl_features_test, ssl_features_ood)
#sp_features_train,  sp_features_test, sp_features_ood= dnorm(sp_features_train, sp_features_test, sp_features_ood)
#cl_features_train,  cl_features_test, cl_features_ood= dnorm(cl_features_train, cl_features_test, cl_features_ood)


fl_features_train = 0.33 * ssl_features_train + 0.33 * sp_features_train + 0.34 * cl_features_train
fl_features_test = 0.33 * ssl_features_test + 0.33 * sp_features_test + 0.34 * cl_features_test
fl_features_ood = 0.33 * ssl_features_ood + 0.33 * sp_features_ood + 0.34 * cl_features_ood
print("new fl_feature_train size is:", fl_features_train.shape)

fl_din, fl_dout, fl_idxin, fl_idxood = get_scores_multi_cluster(fl_features_train, fl_features_test, fl_features_ood, ssl_labels_train)

In [ ]:
auroc = get_eval_results(fl_din, fl_dout)

print("AUROC is:", auroc)


In [ ]:
np.save("./features/ensemble_cifar100/fl_test_score.npy",fl_din)
np.save("./features/ensemble_cifar100/fl_ood_score.npy",fl_dout)
np.save("./features/ensemble_cifar100/fl_ood_minlabels.npy",fl_idxood)
np.save("./features/ensemble_cifar100/fl_test_minlabels.npy",fl_idxin)

In [ ]:
print(sp_dout)
print(sp_din)

In [ ]:
f_change= fl_dout - sp_dout
indices = (-f_change).argsort()[:10]
print(indices)

In [ ]:
plt.figure(0).clf()
plt.plot(sp_features_ood[645])
#     plt.hist(ftrain[60], density=True, bins=100)
plt.show()

plt.figure(0).clf()
plt.plot(cl_features_ood[645])
#     plt.hist(ftrain[60], density=True, bins=100)
plt.show()

idx = 645
plt.figure(0).clf()
plt.plot(ssl_features_ood[idx])
#     plt.hist(ftrain[60], density=True, bins=100)
plt.show()
print((-ssl_features_ood[idx]).argsort()[:10])

In [ ]:
## calculate fpr, tpr, Threshold 

labels = [0] * len(fl_din) + [1] * len(fl_dout)
print("length of labels is:", len(labels))
data = np.concatenate((fl_din, fl_dout))
fpr, tpr, thresh = roc_curve(labels, data)
print(thresh.size)
thresh

In [ ]:
## select threshold that fpr approximates to 0.23
Th = thresh[4315]
print(Th)
index = np.where(sp_din > Th)[0]
print(index.size)
print(index.size/20000)

In [ ]:
# score distritbuion of sl and ssl and ensemble
plt.title("Supervised score distribution")
plt.hist(sp_din, alpha=.6, bins=50, label= 'ID')  # density=False would make counts
plt.hist(sp_dout, alpha=.6, bins=50, label= 'ID')  # density=False would make counts
plt.ylabel('frequency')
plt.xlabel('Score');
plt.show()

plt.title("SuperCon score distribution")
plt.hist(cl_din, alpha=.6, bins=50, label= 'ID')  # density=False would make counts
plt.hist(cl_dout, alpha=.6, bins=50, label= 'ID')  # density=False would make counts
plt.ylabel('frequency')
plt.xlabel('Score');
plt.show()

plt.title("Self-Con score distribution")
plt.hist(ssl_din, alpha=.6, bins=50, label= 'ID')  # density=False would make counts
plt.hist(ssl_dout, alpha=.6, bins=50, label= 'ID')  # density=False would make counts
plt.ylabel('frequency')
plt.xlabel('Score');
plt.show()

plt.title("Ensemble score distribution")
plt.hist(fl_din, alpha=.6, bins=50, label= 'ID')  # density=False would make counts
plt.hist(fl_dout, alpha=.6, bins=50, label= 'ID')  # density=False would make counts
plt.ylabel('frequency')
plt.xlabel('Score');
plt.show()


# plt.hist(ssl_dout, alpha=.6, bins=50)  # density=False would make counts
# plt.hist(fl_dout, alpha=.6, bins=50)  # density=False would make counts
# plt.ylabel('frequency')
# plt.xlabel('Score');
# plt.show()



# plt.hist(ssl_din, alpha=.6, bins=50)  # density=False would make counts
# plt.hist(fl_din, alpha=.6, bins=50)  # density=False would make counts
# plt.ylabel('frequency')
# plt.xlabel('Score');
# plt.show()



plt.title("OOD Sp-Cl-ssl-Ensemble")
plt.hist(sp_dout, alpha=.4, bins=50, label='sp')  # density=False would make counts
plt.hist(cl_dout, alpha=.4, bins=50, label='superCon')
plt.hist(ssl_dout, alpha=.4, bins=50, label='ssl')
plt.hist(fl_dout, alpha=.4, bins=50, label='ensemble')# density=False would make counts
plt.legend()
plt.ylabel('frequency')
plt.xlabel('Score');
plt.show()



# plt.hist(sp_din, alpha=.6, bins=50)  # density=False would make counts
# plt.hist(fl_din, alpha=.6, bins=50)  # density=False would make counts
# plt.ylabel('frequency')
# plt.xlabel('Score');
# plt.show()



plt.title("ID Sp-SuperCon-SSL-Ensemble")
plt.hist(sp_din, alpha=.4, bins=50, label='supervised')  # density=False would make counts
plt.hist(cl_din, alpha=.4, bins=50, label='supervised Contrastive')
plt.hist(ssl_din, alpha=.4, bins=50, label='self-supervised learning')
plt.hist(fl_din, alpha=.4, bins=50, label='ensemble')# density=False would make counts
plt.legend()
plt.ylabel('frequency')
plt.xlabel('Score');
plt.show()


print(ssl_dout.size)
print(fl_dout.size)

In [ ]:
# find a identical precision rate (TP/(TP+FP))
# find a identical fpr 
Th2 = thresh[3200]
print(Th2)
sp_fpr_idx = np.where(sp_din > Th)[0]
fl_fpr_idx = np.where(fl_din > Th2)[0]
print(sp_fpr_idx.size)
print(fl_fpr_idx.size)


sp_fnr_idx = np.where(sp_dout < Th)[0]
fl_fnr_idx = np.where(fl_dout < Th2)[0]
print(sp_fnr_idx.size)
print(fl_fnr_idx.size)
print(sp_fnr_idx[:150])
print(fl_fnr_idx[:150])

In [ ]:
ood_labels = np.load("./supervised_cifar100/labels_ood.npy")

In [ ]:
f_change= fl_dout - sp_dout
indices = (-f_change).argsort()[:10]
print(indices)

In [ ]:
## find sample index that change from TP to FN. 
idx_fn_to_tp1 = np.setdiff1d(fl_fnr_idx, sp_fnr_idx)
print(idx_fn_to_tp1.size)

In [ ]:
## get sample index that change from FN to TP
## get the corresponding label and closest cluster

idx_fn_to_tp = np.setdiff1d(sp_fnr_idx, fl_fnr_idx)
print(idx_fn_to_tp.size)
print(idx_fn_to_tp[:415])
# print(ood_labels[fn_to_tp[:415]])
label_fn_to_tp = ood_labels[idx_fn_to_tp[:415]]
print(label_fn_to_tp)
unique, counts = np.unique(label_fn_to_tp, return_counts=True)
print(np.asarray((unique, counts)).T)
# print(counts)
plt.plot(counts)
plt.ylabel('frequency')
plt.xlabel('class');
plt.show()


##  cluster 
sl_cluster_fn_to_tp = sp_idxood[idx_fn_to_tp]
fl_cluster_fn_to_tp = fl_idxood[idx_fn_to_tp]
# print(sl_cluster_fn_to_tp)
# print(fl_cluster_fn_to_tp)
d_index = np.where(sl_cluster_fn_to_tp != fl_cluster_fn_to_tp)
# print(d_index)
print(sl_cluster_fn_to_tp[d_index])
print(fl_cluster_fn_to_tp[d_index])

print(np.where(label_fn_to_tp == 81))

In [ ]:
## visualize one fn -> tp sample feature


plt.figure(0).clf()
# plt.plot(0.9 * sp_features_ood[50])
plt.hist(0.9 * sp_features_ood[50], alpha=.6, bins=100)
plt.hist(fl_features_ood[60], alpha=.6, bins=100)
plt.show()

plt.figure(1).clf()
# plt.plot(0.9 * sp_features_ood[50])
plt.hist(0.9 * sp_features_ood[6], alpha=.6, bins=100)
plt.hist(fl_features_ood[6], alpha=.6, bins=100)
plt.show()

plt.figure(2).clf()
# plt.plot(0.9 * sp_features_ood[50])
plt.hist(0.9 * sp_features_ood[11], alpha=.6, bins=100)
plt.hist(fl_features_ood[11], alpha=.6, bins=100)
plt.show()
# plt.figure(1).clf()
# # plt.plot(0.1 * ssl_features_ood[50])
# plt.hist(0.1 * ssl_features_ood[50], bins=100)
# plt.show()


# plt.figure(2).clf()
# # plt.plot(0.1 * fl_features_ood[50])
# plt.hist(fl_features_ood[60], bins=100)
# plt.show()

In [ ]:
ix = idx_fn_to_tp[225]
print(ix)

# plt.figure(0).clf()
# plt.plot(0.9 * sp_features_ood[50])
plt.hist(0.9 * sp_features_ood[ix], alpha=.6, bins=15)
plt.hist(fl_features_ood[ix], alpha=.6, bins=15)
plt.show()

In [ ]:
# ## get false negative index and number
# index2 = np.where(sp_dout < Th)[0]
# # print(index[:150])
# print(index2.size)
# # print(ood_labels[:150])

In [ ]:
# ## get false negative index and number
# index2 = np.where(sp_dout < Th)[0]
# # print(index[:150])
# print(index2.size)
# # print(ood_labels[:150])

In [ ]:
# index1 = np.where(fl_dout < Th)[0]
# # print(index1)
# index1.size

In [ ]:
def get_roc_sklearn_plot1(xin1, xood1, xin2, xood2, xin3, xood3):
    
    labels1 = [0] * len(xin1) + [1] * len(xood1)
    print("length of labels is:", len(labels1))
    data1 = np.concatenate((xin1, xood1))
    labels2 = [0] * len(xin2) + [1] * len(xood2)
    data2 = np.concatenate((xin2, xood2))
    labels3 = [0] * len(xin3) + [1] * len(xood3)
    data3 = np.concatenate((xin3, xood3))
    
    plt.figure(0).clf()
    fpr, tpr, thresh = roc_curve(labels1, data1)
    print("size of threshold is :", thresh)
    plt.plot(fpr,tpr,label="SL")
    print("size of fpr is :", fpr)
    print("size of tpr is :", tpr)
    
    fpr, tpr, thresh = roc_curve(labels2, data2)
    plt.plot(fpr,tpr,label="SSL")
    
    
    fpr, tpr, thresh = roc_curve(labels3, data3)
    plt.plot(fpr,tpr,label="Ensemble")
    
get_roc_sklearn_plot1(sp_din, sp_dout, ssl_din, ssl_dout, fl_din, fl_dout)

In [ ]:
# fn_to_tp = np.setdiff1d(index, index1)
# print(fn_to_tp[:150])
# print(ood_labels[fn_to_tp[:150]])
# print(ood_labels[50])

In [ ]:
## visualize one fn -> tp sample feature

plt.figure(0).clf()
plt.plot(0.9 * sp_features_ood[50])
#     plt.hist(ftrain[60], density=True, bins=100)
plt.show()

plt.figure(1).clf()
plt.plot(0.1 * ssl_features_ood[50])
#     plt.hist(ftrain[60], density=True, bins=100)
plt.show()


plt.figure(1).clf()
plt.plot(0.1 * fl_features_ood[50])
#     plt.hist(ftrain[60], density=True, bins=100)
plt.show()


In [ ]:
## visualize one fn -> tp sample feature

plt.figure(0).clf()
plt.plot(0.33 * sp_features_ood[257])
#     plt.hist(ftrain[60], density=True, bins=100)
plt.show()

plt.figure(1).clf()
plt.plot(0.33 * ssl_features_ood[257])
#     plt.hist(ftrain[60], density=True, bins=100)
plt.show()

plt.figure(2).clf()
plt.plot(0.34 * cl_features_ood[257])
#     plt.hist(ftrain[60], density=True, bins=100)
plt.show()


plt.figure(3).clf()
plt.plot(0.1 * fl_features_ood[257])
#     plt.hist(ftrain[60], density=True, bins=100)
plt.show()

In [ ]:
## find the largest values in ssl feature

ssl_idx = (-ssl_features_ood[6]).argsort()[:10]
ssl_idx

In [ ]:
sp_features_ood[6][986]

In [ ]:
ssl_idx = (-ssl_features_ood[17]).argsort()[:10]
ssl_idx

In [ ]:
a = [1, 2, 3]
b = [1]